In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from functions import *

In [3]:
from numba import njit

In [4]:
@njit
def rk_solver_mat(a: float, b: float, k: float,
                  α: np.ndarray, β: float, γ: np.ndarray,
                  x: np.ndarray, y: np.ndarray, z: np.ndarray,
                  n_steps: int, transient_drop: int, batchs=1, h=0.001) -> np.ndarray:
    trajectory = np.empty((batchs, n_steps - transient_drop, 3), dtype=np.float32)

    idx = 0
    for t in range(0, n_steps):
        k1 = dx_dt(a, b, k, α, x, y)
        l1 = dy_dt(k, x, y, z)
        m1 = dz_dt(k, β, γ, y, z)

        k2 = dx_dt(a, b, k, α, x + 0.5 * h * k1, y + 0.5 * h * l1)
        l2 = dy_dt(k, x + 0.5 * h * k1, y + 0.5 * h * l1, z + 0.5 * h * m1)
        m2 = dz_dt(k, β, γ, y + 0.5 * h * l1, z + 0.5 * h * m1)

        k3 = dx_dt(a, b, k, α, x + 0.5 * h * k2, y + 0.5 * h * l2)
        l3 = dy_dt(k, x + 0.5 * h * k2, y + 0.5 * h * l2, z + 0.5 * h * m2)
        m3 = dz_dt(k, β, γ, y + 0.5 * h * l2, z + 0.5 * h * m2)

        k4 = dx_dt(a, b, k, α, x + h * k3, y + h * l3)
        l4 = dy_dt(k, x + h * k3, y + h * l3, z + h * m3)
        m4 = dz_dt(k, β, γ, y + h * l3, z + h * m3)

        x = x + 1 / 6 * (k1 + 2 * k2 + 2 * k3 + k4) * h
        y = y + 1 / 6 * (l1 + 2 * l2 + 2 * l3 + l4) * h
        z = z + 1 / 6 * (m1 + 2 * m2 + 2 * m3 + m4) * h
        
        if t >= transient_drop:
            trajectory[:, idx, 0] = x
            trajectory[:, idx, 1] = y
            trajectory[:, idx, 2] = z
            idx += 1

    return trajectory

In [5]:
alpha_gamma_mat = np.load('../Data/alpha_gamma.npy')

In [8]:
β = 1000.0
a = -8/7
b = -5/7
k = 1
init_cond = [1.1, 0.12, 0.01]
batch = 1200
# batch = 1
crop = alpha_gamma_mat[0, :batch]
α = crop[:, 0]
γ = crop[:, 1]

In [10]:
n_steps, transient_drop = transient(1)
x_init = np.array([init_cond[0]]*batch)
y_init = np.array([init_cond[1]]*batch)
z_init = np.array([init_cond[2]]*batch)
trajs = rk_solver_mat(a, b, k, α, β, γ, x_init, y_init, z_init, n_steps, transient_drop, batch)

In [13]:
idx = np.random.randint(0, batch)

In [16]:
single_trajectory = rk_solver(a, b, k, α[idx], β, γ[idx], init_cond[0], init_cond[1], init_cond[2], n_steps, transient_drop, 0.001)

In [21]:
print(np.array_equal(trajs[idx], single_trajectory))

True
